## El conjunto de datos CelebA 

Utilizaremos el conjunto de datos CelebFaces Attributes para entrenar nuestro modelo autocodificador variacional. Se puede encontrar en Kaggle, [CelebA](https://www.kaggle.com/datasets/jessicali9530/celeba-dataset)

In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np

print(f'Versión de TensorFlow: {tf.__version__}')
print(f"Dispositivo de entrenamiento: {tf.config.list_physical_devices('GPU')}")

Versión de TensorFlow: 2.10.0
Dispositivo de entrenamiento: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [ ]:
train_data = tf.keras.utils.image_dataset_from_directory(
    "../datasets/celebA/img",
    labels=None,
    color_mode='rgb',
    image_size=(64, 64),
    batch_size=128,
    shuffle=True,
    seed=42,
    interpolation='bilinear'
)

In [ ]:
def preprocess(img):
    return tf.cast(img, "float32") / 255.0

train = train_data.map(lambda x: preprocess(x))